In [1]:
!pip install squarify

In [0]:
import re
import string
from collections import Counter
import squarify
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
from spacy.tokenizer import Tokenizer

from bs4 import BeautifulSoup
import html as ihtml

import requests
import sqlite3

count_df = pd.DataFrame(columns=['job_count', 'title_name', 'city_name'])

In [0]:
def extract_job_title(soup):
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

In [0]:
def extract_company(soup):
    companies = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name="span", attrs={"class":"result-linl-source"})
            for span in sec_try:
                companies.append(span.text.strip())
    return(companies)

In [0]:
def extract_location(soup):
    locations = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            locations.append(div.find(name="span", attrs={"class":"location"}).text)
        except:
            locations.append("None")
    return(locations)

In [0]:
def extract_salary(soup):
    salaries = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            salaries.append(div.find(name="span", attrs={"class":"salaryText"}).text.replace("\n",""))
        except:
            salaries.append("None")
    return(salaries)

In [0]:
def extract_job_title(soup):
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

In [0]:
def extract_url(soup):
    urls = []
    for div1 in soup.find_all(name="div", attrs={"class":"row"}):
        for div2 in div1.find_all(name="div", attrs={"class":"title"}):
            for a in div2.find_all(name="a", href=True):
                urls.append(a['href'])
    return(urls)

def extract_desc(urls):
    descs = []
    for url in urls:
        full_url = "https://www.indeed.com" + url
        detail_page = requests.get(full_url)
        detail_soup = BeautifulSoup(detail_page.text, "html.parser")
        
        for div in detail_soup.find_all(name="div", attrs={"id":"jobDescriptionText"}):
            descs.append(div.text)
    return(descs)

In [0]:
def extract_count(soup):
    for div in soup.find_all(name="div", attrs={"id":"searchCountPages"}):
        temp_str = div.text.replace(" ", "")
        temp_count_str = re.search('of(.*)jobs', temp_str)
        count_str = re.sub('[^0-9]','', temp_count_str.group(1))
        count = int(count_str)
    return(count)

In [0]:
def extract_list(title_name, city_name, st_name):
    max_results = 100
    columns = ["city", "job_title", "company", "location", "salary", "description"]

    city_url = "https://www.indeed.com/jobs?q=" + title_name + \
               "&l=" + city_name + "%2C+" + st_name
    page = requests.get(city_url)
    soup = BeautifulSoup(page.text, "html.parser")
    max_counter = extract_count(soup)
    print(max_counter, title_name, city_name)
   
    job_title_list = []
    company_list = []
    location_list = []
    salary_list = []
    desc_list = []

    for start in range(0, max_results, 10):
        city_url = "https://www.indeed.com/jobs?q=" + title_name + \
                   "&l=" + city_name + "%2C+" + st_name + \
                   "&start=" + str(start)
        page = requests.get(city_url)
        soup = BeautifulSoup(page.text, "html.parser")

        job_title_list.extend(extract_job_title(soup))
        company_list.extend(extract_company(soup))
        location_list.extend(extract_location(soup))
        salary_list.extend(extract_salary(soup))
        add_urls = extract_url(soup)
        desc_list.extend(extract_desc(add_urls))

    return job_title_list, company_list, location_list, salary_list, desc_list, max_counter

def job_db(title, city, st):   
    j_list, c_list, l_list, s_list, d_list, m_counter = extract_list(title, city, st)     
    temp_df = pd.DataFrame(list(zip(j_list, c_list, l_list, s_list, d_list)), 
                          columns = ['job_title', 'company', 'location', 'salary', 'description'])
    temp_df['counts'] = m_counter
    return temp_df

In [11]:
temp1_df = job_db('data+scientist', 'San+Jose', 'CA')
temp1_df['city'] = 'San Jose'
temp2_df = job_db('data+scientist', 'San+Francisco', 'CA')
temp2_df['city'] = 'San Francisco'
temp3_df = job_db('data+scientist', 'Seattle', 'WA')
temp3_df['city'] = 'Seattle'
temp4_df = job_db('data+scientist', 'Washington', 'DC')
temp4_df['city'] = 'Washington'
temp5_df = job_db('data+scientist', 'New+York', 'NY')
temp5_df['city'] = 'New York'
temp6_df = job_db('data+scientist', 'Baltimore', 'MD')
temp6_df['city'] = 'Baltimore'
temp7_df = job_db('data+scientist', 'Boulder', 'CO')
temp7_df['city'] = 'Boulder'
temp8_df = job_db('data+scientist', 'San+Diego', 'CA')
temp8_df['city'] = 'San Diego'
temp9_df = job_db('data+scientist', 'Denver', 'CO')
temp9_df['city'] = 'Denver'
temp10_df = job_db('data+scientist', 'Huntsville', 'AL')
temp10_df['city'] = 'Huntsville'
temp11_df = job_db('data+scientist', 'Colorado+Springs', 'CO')
temp11_df['city'] = 'Colorado Springs'
temp12_df = job_db('data+scientist', 'Houston', 'TX')
temp12_df['city'] = 'Houston'
temp13_df = job_db('data+scientist', 'Trenton', 'NJ')
temp13_df['city'] = 'Trenton'
temp14_df = job_db('data+scientist', 'Dallas', 'TX')
temp14_df['city'] = 'Dallas'
temp15_df = job_db('data+scientist', 'Columbus', 'OH')
temp15_df['city'] = 'Columbus'
temp16_df = job_db('data+scientist', 'Austin', 'TX')
temp16_df['city'] = 'Austin'
temp17_df = job_db('data+scientist', 'Philadelphia', 'PA')
temp17_df['city'] = 'Philadelphia'
temp18_df = job_db('data+scientist', 'Durham', 'NC')
temp18_df['city'] = 'Durham'
temp19_df = job_db('data+scientist', 'Raleigh', 'NC')
temp19_df['city'] = 'Raleigh'
temp20_df = job_db('data+scientist', 'Atlanta', 'GA')
temp20_df['city'] = 'Atlanta'

data_scientist_df = pd.concat([temp1_df, temp2_df, temp3_df, temp4_df, temp5_df,
                       temp6_df, temp7_df, temp8_df, temp9_df, temp10_df,
                       temp11_df, temp12_df, temp13_df, temp14_df, temp15_df,
                       temp16_df, temp17_df, temp18_df, temp19_df, temp20_df], ignore_index=True)

print(data_scientist_df.shape)
count_df.head()

1319 data+scientist San+Jose
937 data+scientist San+Francisco
1106 data+scientist Seattle
1181 data+scientist Washington
818 data+scientist New+York
165 data+scientist Baltimore
161 data+scientist Boulder
164 data+scientist San+Diego
180 data+scientist Denver
22 data+scientist Huntsville
6 data+scientist Colorado+Springs
375 data+scientist Houston
92 data+scientist Trenton
257 data+scientist Dallas
74 data+scientist Columbus
185 data+scientist Austin
215 data+scientist Philadelphia
145 data+scientist Durham
147 data+scientist Raleigh
257 data+scientist Atlanta
(3348, 7)


,job_count,title_name,city_name


In [12]:
temp1_df = job_db('web+developer', 'San+Jose', 'CA')
temp1_df['city'] = 'San Jose'
temp2_df = job_db('web+developer', 'San+Francisco', 'CA')
temp2_df['city'] = 'San Francisco'
temp3_df = job_db('web+developer', 'Seattle', 'WA')
temp3_df['city'] = 'Seattle'
temp4_df = job_db('web+developer', 'Washington', 'DC')
temp4_df['city'] = 'Washington'
temp5_df = job_db('web+developer', 'New+York', 'NY')
temp5_df['city'] = 'New York'
temp6_df = job_db('web+developer', 'Baltimore', 'MD')
temp6_df['city'] = 'Baltimore'
temp7_df = job_db('web+developer', 'Boulder', 'CO')
temp7_df['city'] = 'Boulder'
temp8_df = job_db('web+developer', 'San+Diego', 'CA')
temp8_df['city'] = 'San Diego'
temp9_df = job_db('web+developer', 'Denver', 'CO')
temp9_df['city'] = 'Denver'
temp10_df = job_db('web+developer', 'Huntsville', 'AL')
temp10_df['city'] = 'Huntsville'
temp11_df = job_db('web+developer', 'Colorado+Springs', 'CO')
temp11_df['city'] = 'Colorado Springs'
temp12_df = job_db('web+developer', 'Houston', 'TX')
temp12_df['city'] = 'Houston'
temp13_df = job_db('web+developer', 'Trenton', 'NJ')
temp13_df['city'] = 'Trenton'
temp14_df = job_db('web+developer', 'Dallas', 'TX')
temp14_df['city'] = 'Dallas'
temp15_df = job_db('web+developer', 'Columbus', 'OH')
temp15_df['city'] = 'Columbus'
temp16_df = job_db('web+developer', 'Austin', 'TX')
temp16_df['city'] = 'Austin'
temp17_df = job_db('web+developer', 'Philadelphia', 'PA')
temp17_df['city'] = 'Philadelphia'
temp18_df = job_db('web+developer', 'Durham', 'NC')
temp18_df['city'] = 'Durham'
temp19_df = job_db('web+developer', 'Raleigh', 'NC')
temp19_df['city'] = 'Raleigh'
temp20_df = job_db('web+developer', 'Atlanta', 'GA')
temp20_df['city'] = 'Atlanta'

web_developer_df = pd.concat([temp1_df, temp2_df, temp3_df, temp4_df, temp5_df,
                              temp6_df, temp7_df, temp8_df, temp9_df, temp10_df,
                              temp11_df, temp12_df, temp13_df, temp14_df, temp15_df,
                              temp16_df, temp17_df, temp18_df, temp19_df, temp20_df], ignore_index=True)
print(web_developer_df.shape)

1417 web+developer San+Jose
1417 web+developer San+Francisco
1422 web+developer Seattle
3433 web+developer Washington
1400 web+developer New+York
880 web+developer Baltimore
440 web+developer Boulder
384 web+developer San+Diego
597 web+developer Denver
113 web+developer Huntsville
68 web+developer Colorado+Springs
367 web+developer Houston
227 web+developer Trenton
778 web+developer Dallas
224 web+developer Columbus
652 web+developer Austin
533 web+developer Philadelphia
377 web+developer Durham
360 web+developer Raleigh
618 web+developer Atlanta
(3644, 7)


In [13]:
temp1_df = job_db('ux+designer', 'San+Jose', 'CA')
temp1_df['city'] = 'San Jose'
temp2_df = job_db('ux+designer', 'San+Francisco', 'CA')
temp2_df['city'] = 'San Francisco'
temp3_df = job_db('ux+designer', 'Seattle', 'WA')
temp3_df['city'] = 'Seattle'
temp4_df = job_db('ux+designer', 'Washington', 'DC')
temp4_df['city'] = 'Washington'
temp5_df = job_db('ux+designer', 'New+York', 'NY')
temp5_df['city'] = 'New York'
temp6_df = job_db('ux+designer', 'Baltimore', 'MD')
temp6_df['city'] = 'Baltimore'
temp7_df = job_db('ux+designer', 'Boulder', 'CO')
temp7_df['city'] = 'Boulder'
temp8_df = job_db('ux+designer', 'San+Diego', 'CA')
temp8_df['city'] = 'San Diego'
temp9_df = job_db('ux+designer', 'Denver', 'CO')
temp9_df['city'] = 'Denver'
temp10_df = job_db('ux+designer', 'Huntsville', 'AL')
temp10_df['city'] = 'Huntsville'
temp11_df = job_db('ux+designer', 'Colorado+Springs', 'CO')
temp11_df['city'] = 'Colorado Springs'
temp12_df = job_db('ux+designer', 'Houston', 'TX')
temp12_df['city'] = 'Houston'
temp13_df = job_db('ux+designer', 'Trenton', 'NJ')
temp13_df['city'] = 'Trenton'
temp14_df = job_db('ux+designer', 'Dallas', 'TX')
temp14_df['city'] = 'Dallas'
temp15_df = job_db('ux+designer', 'Columbus', 'OH')
temp15_df['city'] = 'Columbus'
temp16_df = job_db('ux+designer', 'Austin', 'TX')
temp16_df['city'] = 'Austin'
temp17_df = job_db('ux+designer', 'Philadelphia', 'PA')
temp17_df['city'] = 'Philadelphia'
temp18_df = job_db('ux+designer', 'Durham', 'NC')
temp18_df['city'] = 'Durham'
temp19_df = job_db('ux+designer', 'Raleigh', 'NC')
temp19_df['city'] = 'Raleigh'
temp20_df = job_db('ux+designer', 'Atlanta', 'GA')
temp20_df['city'] = 'Atlanta'

ux_designer_df = pd.concat([temp1_df, temp2_df, temp3_df, temp4_df, temp5_df,
                            temp6_df, temp7_df, temp8_df, temp9_df, temp10_df,
                            temp11_df, temp12_df, temp13_df, temp14_df, temp15_df,
                            temp16_df, temp17_df, temp18_df, temp19_df, temp20_df], ignore_index=True)
print(ux_designer_df.shape)

634 ux+designer San+Jose
874 ux+designer San+Francisco
952 ux+designer Seattle
522 ux+designer Washington
875 ux+designer New+York
89 ux+designer Baltimore
132 ux+designer Boulder
107 ux+designer San+Diego
147 ux+designer Denver
8 ux+designer Huntsville
6 ux+designer Colorado+Springs
82 ux+designer Houston
41 ux+designer Trenton
177 ux+designer Dallas
47 ux+designer Columbus
274 ux+designer Austin
145 ux+designer Philadelphia
81 ux+designer Durham
81 ux+designer Raleigh
248 ux+designer Atlanta
(3056, 7)


In [14]:
temp1_df = job_db('ios+developer', 'San+Jose', 'CA')
temp1_df['city'] = 'San Jose'
temp2_df = job_db('ios+developer', 'San+Francisco', 'CA')
temp2_df['city'] = 'San Francisco'
temp3_df = job_db('ios+developer', 'Seattle', 'WA')
temp3_df['city'] = 'Seattle'
temp4_df = job_db('ios+developer', 'Washington', 'DC')
temp4_df['city'] = 'Washington'
temp5_df = job_db('ios+developer', 'New+York', 'NY')
temp5_df['city'] = 'New York'
temp6_df = job_db('ios+developer', 'Baltimore', 'MD')
temp6_df['city'] = 'Baltimore'
temp7_df = job_db('ios+developer', 'Boulder', 'CO')
temp7_df['city'] = 'Boulder'
temp8_df = job_db('ios+developer', 'San+Diego', 'CA')
temp8_df['city'] = 'San Diego'
temp9_df = job_db('ios+developer', 'Denver', 'CO')
temp9_df['city'] = 'Denver'
temp10_df = job_db('ios+developer', 'Huntsville', 'AL')
temp10_df['city'] = 'Huntsville'
temp11_df = job_db('ios+developer', 'Colorado+Springs', 'CO')
temp11_df['city'] = 'Colorado Springs'
temp12_df = job_db('ios+developer', 'Houston', 'TX')
temp12_df['city'] = 'Houston'
temp13_df = job_db('ios+developer', 'Trenton', 'NJ')
temp13_df['city'] = 'Trenton'
temp14_df = job_db('ios+developer', 'Dallas', 'TX')
temp14_df['city'] = 'Dallas'
temp15_df = job_db('ios+developer', 'Columbus', 'OH')
temp15_df['city'] = 'Columbus'
temp16_df = job_db('ios+developer', 'Austin', 'TX')
temp16_df['city'] = 'Austin'
temp17_df = job_db('ios+developer', 'Philadelphia', 'PA')
temp17_df['city'] = 'Philadelphia'
temp18_df = job_db('ios+developer', 'Durham', 'NC')
temp18_df['city'] = 'Durham'
temp19_df = job_db('ios+developer', 'Raleigh', 'NC')
temp19_df['city'] = 'Raleigh'
temp20_df = job_db('ios+developer', 'Atlanta', 'GA')
temp20_df['city'] = 'Atlanta'

ios_developer_df = pd.concat([temp1_df, temp2_df, temp3_df, temp4_df, temp5_df,
                              temp6_df, temp7_df, temp8_df, temp9_df, temp10_df,
                              temp11_df, temp12_df, temp13_df, temp14_df, temp15_df,
                              temp16_df, temp17_df, temp18_df, temp19_df, temp20_df], ignore_index=True)
print(ios_developer_df.shape)

757 ios+developer San+Jose
463 ios+developer San+Francisco
482 ios+developer Seattle
351 ios+developer Washington
315 ios+developer New+York
83 ios+developer Baltimore
107 ios+developer Boulder
90 ios+developer San+Diego
132 ios+developer Denver
15 ios+developer Huntsville
6 ios+developer Colorado+Springs
53 ios+developer Houston
24 ios+developer Trenton
153 ios+developer Dallas
28 ios+developer Columbus
174 ios+developer Austin
87 ios+developer Philadelphia
48 ios+developer Durham
47 ios+developer Raleigh
135 ios+developer Atlanta
(3224, 7)


In [0]:
data_scientist_df['job'] = 'data scientist'
web_developer_df['job'] = 'web developer'
ux_designer_df['job'] = 'ux designer'
ios_developer_df['job'] = 'ios developer'
indeed_df = pd.concat([data_scientist_df, web_developer_df, ux_designer_df, ios_developer_df], ignore_index=True)
indeed_df = indeed_df.drop_duplicates(keep=False) 
indeed_df = indeed_df.reset_index(drop=True)

In [16]:
indeed_df.head()

,job_title,company,location,salary,description,counts,city,job
0,Data Scientist,Pactera,None,None,Looking fora Data Scientis/Architect who has 8...,1319,San Jose,data scientist
1,Data scientist - Global Sales,PayPal,None,None,Responsibilities:\n• Provide business requirem...,1319,San Jose,data scientist
2,Data Scientist,Palo Verde Consulting,"Campbell, CA 95008 (Central Campbell area)","$150,000 - $210,000 a year","Job Title: Data ScientistLocation: Campbell, C...",1319,San Jose,data scientist
3,Data Scientist,Spry Health,"Palo Alto, CA","$100,000 - $135,000 a year",Spry Health’s mission is to build the world’s ...,1319,San Jose,data scientist
4,Data Scientist (All Levels) - Santa Clara,LeanTaaS,"Santa Clara, CA 95050",None,Help build technology that saves lives!\n\nWe'...,1319,San Jose,data scientist


In [17]:
def convert_salary(sal_str, s_flag):
    
    sal_split = re.findall(r'\d+', sal_str.replace(",", ""))
    
    if len(sal_split) == 2:
        low_salary = int(sal_split[0])
        high_salary = int(sal_split[1])
    else:
        low_salary = None
        high_salary = None
    
    if s_flag == 'l':
        salary = low_salary
    else:
        salary = high_salary
    
    return salary

indeed_df['low_salary'] = indeed_df.apply(lambda x: convert_salary(x['salary'], 'l'), axis=1)
indeed_df['high_salary'] = indeed_df.apply(lambda x: convert_salary(x['salary'], 'h'), axis=1)
indeed_df = indeed_df.drop(columns=['salary'])

indeed_df.head()                                      

,job_title,company,location,description,counts,city,job,low_salary,high_salary
0,Data Scientist,Pactera,None,Looking fora Data Scientis/Architect who has 8...,1319,San Jose,data scientist,NaN,NaN
1,Data scientist - Global Sales,PayPal,None,Responsibilities:\n• Provide business requirem...,1319,San Jose,data scientist,NaN,NaN
2,Data Scientist,Palo Verde Consulting,"Campbell, CA 95008 (Central Campbell area)","Job Title: Data ScientistLocation: Campbell, C...",1319,San Jose,data scientist,150000.0,210000.0
3,Data Scientist,Spry Health,"Palo Alto, CA",Spry Health’s mission is to build the world’s ...,1319,San Jose,data scientist,100000.0,135000.0
4,Data Scientist (All Levels) - Santa Clara,LeanTaaS,"Santa Clara, CA 95050",Help build technology that saves lives!\n\nWe'...,1319,San Jose,data scientist,NaN,NaN


In [18]:
indeed_df.tail()

,job_title,company,location,description,counts,city,job,low_salary,high_salary
4851,Senior Full Stack Developer,Revel Systems,"Atlanta, GA",Company Description\n\nRevel was founded with ...,135,Atlanta,ios developer,NaN,NaN
4852,Product Manager / Business Analyst,Worry Free Labs,"Atlanta, GA",KEY RESPONSIBILITIES:\nMeeting with business p...,135,Atlanta,ios developer,NaN,NaN
4853,Account Executive,MacStadium,"Atlanta, GA 30305 (Buckhead area)",Overview\nMacStadium is the leading managed Ma...,135,Atlanta,ios developer,NaN,NaN
4854,"Innovation Software Architect I, Full Stack",InComm,"Atlanta, GA 30303 (Five Points area)",Overview\nLeveraging deep integrations into re...,135,Atlanta,ios developer,NaN,NaN
4855,UX/UI Designer,OutSystems,"Atlanta, GA 30319","Job description, low-code style:\nAs the #1 lo...",135,Atlanta,ios developer,NaN,NaN


In [19]:
count_df.head()

,job_count,title_name,city_name


In [0]:
def clean_text(text):
    text = text.replace('\\n', ' ')               # remove newline
    text = BeautifulSoup(text, "lxml").get_text() # remove html
    text = text.replace('/', ' ')                 # remove forward slashes
    text = re.sub(r'[^a-zA-Z ^0-9]', '', text)    # letters and numbers only
    text = text.lower()                           # lower case
    text = re.sub(r'(x.[0-9])', '', text)         # remove special characters
    return text

#indeed_df['description'] = indeed_df.apply(lambda x: clean_text(x['description']), axis=1)

#indeed_df.head()

In [21]:
indeed_df['description'][0]

'Looking fora Data Scientis/Architect who has 8 yrs of exp in data design, data modelling, data flow, analytics in supply chain domain.\n\nHere’s the detail JD:\n\nExpert programming skills in Python, R\nExperience in writing code for various Machine learning algorithms for classification, clustering, forecasting, regression, Neural networks and Deep Learning\nHands-on experience with modern enterprise data architectures and data toolsets (ex: data warehouse, data marts, data lake, 3NF and dimensional models, modeling tools, profiling tools)\nStrong knowledge of Supply Chain domain, preferably in the hi-tech industry\nStrong problem solving and abstract thinking skills\nKnowledge of data architecture and design patterns and the ability to apply them\nAbility to conceptualize and articulate ideas clearly and concisely\nExcellent communication, presentation and interpersonal skills'

In [22]:
indeed_df['description'][4]

"Help build technology that saves lives!\n\nWe're a fast growing healthcare tech company that uses data science and machine learning to make our hospitals more efficient.\n\n\nMore than 65 hospital systems including some of the nation's largest rely on our products to improve patient access and lower wait times.\nWe've raised $70+ million dollars from prominent investors in healthcare.\nOur team includes veteran executives and the brightest minds from Google, McKinsey, Stanford, MIT, Duke, Berkeley, UIUC, and more.\nCBInsights recently named us among the top 100 AI companies in the world.\n\nWe're looking for extraordinary data scientists for our Santa Clara, California office. The Data Scientist role involves working on all stages of the data science pipeline, from acquiring and munging data, selecting appropriate models and algorithms and/or deriving custom algorithms, to testing and evaluating these algorithms and models and incorporating them into commercial products. She/He will w

In [0]:
# convert to sqlite3
conn = sqlite3.connect('techsearch.sqlite3')
curs = conn.cursor()
curs.execute('drop table if exists listings')
indeed_df.to_sql('listings', con=conn)
indeed_df.to_csv('/content/techsearch.csv')